In [37]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import joblib 
import gzip

In [38]:
Modelo1 = pd.read_csv('./EDA/Data-EDA/steam_games_EDA.csv.gz',compression='gzip')

In [39]:
Modelo1

,genres,app_name,specs,price,id,developer,año
0,Strategy,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
1,Action,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
2,Indie,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
3,Casual,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
4,Simulation,Lost summoner kitty,Single-player,4.99,761140.0,Kotoshiro,2018
...,...,...,...,...,...,...,...
784882,VR,Maze run vr,HTC Vive,4.99,681550.0,Otros,2016
784883,VR,Maze run vr,Oculus Rift,4.99,681550.0,Otros,2016
784884,VR,Maze run vr,Tracked Motion Controllers,4.99,681550.0,Otros,2016
784885,VR,Maze run vr,Standing,4.99,681550.0,Otros,2016


In [40]:
# Selección de columnas relevantes
Modelo1 = Modelo1[['id', 'app_name', 'genres', 'developer', 'specs']]

In [41]:
# Limpieza y preprocesamiento de texto
Modelo1['combined_features'] = Modelo1['genres'] + ' ' + Modelo1['developer'] + ' ' + Modelo1['specs']

C:\Users\belen\AppData\Local\Temp\ipykernel_12800\325279200.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Modelo1['combined_features'] = Modelo1['genres'] + ' ' + Modelo1['developer'] + ' ' + Modelo1['specs']


In [42]:
Modelo1.shape

(784887, 6)

In [43]:
Modelo1.isna().sum()

id                     0
app_name               0
genres                 0
developer            209
specs                  0
combined_features    209
dtype: int64

In [44]:
Modelo1=Modelo1.dropna()

Se agrupa la columna 'id'

In [45]:
Modelo1 = Modelo1.groupby('id').first().reset_index()

Se guardan los 'id' distintos a 0

In [46]:
Modelo1 = Modelo1[Modelo1['id']!=0]

In [47]:
Modelo1['app_name'].nunique()

31275

Nos quedamos con una muestra de 3500 filas

In [48]:
Modelo1 = Modelo1.sample(n=3500)

In [49]:
Modelo1.shape

(3500, 6)

In [50]:
Modelo1.reset_index(drop=True, inplace=True)

In [51]:
Modelo1

,id,app_name,genres,developer,specs,combined_features
0,413460.0,Draft day sports college basketball 3,Strategy,Viva media,Single-player,Strategy Viva media Single-player
1,754120.0,Ninja stealth 3,Strategy,Sc jogos,Single-player,Strategy Sc jogos Single-player
2,258658.0,Train simulator: nj transit® alp-46 loco add-on,Simulation,Dovetail games - trains,Single-player,Simulation Dovetail games - trains Single-player
3,689120.0,Scuffle scoundrels,Early Access,Moped games,Single-player,Early Access Moped games Single-player
4,605954.0,Blue medusa - awesomenauts droppod,Strategy,Ronimo games,Single-player,Strategy Ronimo games Single-player
...,...,...,...,...,...,...
3495,676730.0,Behind the door,Indie,Dio alias games,Single-player,Indie Dio alias games Single-player
3496,745120.0,Pinball,Indie,Bakno games,Single-player,Indie Bakno games Single-player
3497,233370.0,The raven - legacy of a master thief,Adventure,Thq nordic,Single-player,Adventure Thq nordic Single-player
3498,279720.0,The i of the dragon,RPG,Topware interactive,Single-player,RPG Topware interactive Single-player


Convertimos la columna 'id' a tipo de dato entero

In [52]:
Modelo1['id'] = Modelo1['id'].astype(int)

In [53]:
Modelo1

,id,app_name,genres,developer,specs,combined_features
0,413460,Draft day sports college basketball 3,Strategy,Viva media,Single-player,Strategy Viva media Single-player
1,754120,Ninja stealth 3,Strategy,Sc jogos,Single-player,Strategy Sc jogos Single-player
2,258658,Train simulator: nj transit® alp-46 loco add-on,Simulation,Dovetail games - trains,Single-player,Simulation Dovetail games - trains Single-player
3,689120,Scuffle scoundrels,Early Access,Moped games,Single-player,Early Access Moped games Single-player
4,605954,Blue medusa - awesomenauts droppod,Strategy,Ronimo games,Single-player,Strategy Ronimo games Single-player
...,...,...,...,...,...,...
3495,676730,Behind the door,Indie,Dio alias games,Single-player,Indie Dio alias games Single-player
3496,745120,Pinball,Indie,Bakno games,Single-player,Indie Bakno games Single-player
3497,233370,The raven - legacy of a master thief,Adventure,Thq nordic,Single-player,Adventure Thq nordic Single-player
3498,279720,The i of the dragon,RPG,Topware interactive,Single-player,RPG Topware interactive Single-player


In [54]:
Modelo1['app_name'].value_counts()

app_name
See no evil                                             2
Detective butler: maiden voyage murder - soundtrack     1
Greed corp                                              1
Melody                                                  1
Project abyss - art & music collection                  1
                                                       ..
Bleach the movie: hell verse                            1
Antboy                                                  1
Space harrier™ ii                                       1
Arraynium                                               1
Fantasy grounds - top down tokens - vietnam soldiers    1
Name: count, Length: 3499, dtype: int64

In [55]:
with gzip.open('./Modelo1.csv.gz', 'wb') as f:
    Modelo1.to_csv(f, index=False, encoding='utf-8')

Si es un sistema de recomendación item-item:

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado.

Se preparan los datos de la columna 'combined_features' para ser utilizados en un modelo de machine learning al convertirlos en una matriz de recuento de términos

In [56]:
CV = CountVectorizer()
converted_metrix = CV.fit_transform(Modelo1['combined_features'])

In [57]:
cosine_similarity = cosine_similarity(converted_metrix)

In [58]:
# guardo la matriz

import joblib #pickle es alternativa

joblib.dump(cosine_similarity, 'Modelo1.pkl', compress=1)

['Modelo1.pkl']

In [59]:
import joblib
with open('Modelo1.pkl', 'rb') as file:
        modelo = joblib.load(file)


La siguiente función toma un índice de un juego, calcula la similitud del coseno con respecto a otros juegos, y devuelve los nombres de los 5 juegos más similares.


In [64]:
def recomendacion3(item_id:int):
    #Cargar el modelo entrenado desde el archivo pickle
    with open('Modelo1.pkl', 'rb') as file:
       modelo = joblib.load(file)

    Modelo1 = pd.read_csv('./Modelo1.csv.gz', compression='gzip')
    
    if item_id not in Modelo1['id'].tolist():
       return {"Respuesta": "No se encontraron resultados para la búsqueda realizada"}

    def get_recommendations(idx, cosine_sim=modelo):
       idx = Modelo1[Modelo1['id'] == item_id].index[0]
       sim_scores = list(enumerate(cosine_sim[idx]))
       sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
       sim_scores = sim_scores[1:6]  # Top 5 juegos similares
       game_indices = [i[0] for i in sim_scores]
       return Modelo1['app_name'].iloc[game_indices].tolist()

    #Obtener el índice del item_id
   # 

    recommendations = get_recommendations(item_id)
    return {"Recomendaciones": recommendations}


In [65]:
recomendacion3(745120)

{'Recomendaciones': ['Yatzy',
  'Pinball',
  'Mall empire',
  'Awesomenauts - abyssal swiggins skin',
  'Sidewords']}

Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [66]:
reviews2 = pd.read_csv('./Data/Data-Limpia/user_reviews_limpio.csv.gz',compression='gzip')

In [67]:
steams2 = pd.read_csv('./EDA/Data-EDA/steam_games_EDA.csv.gz',compression='gzip')

Se realiza un merge de los DF reviews2 y steams2

In [68]:
Modelo2 = pd.merge(reviews2, steams2, left_on='item_id', right_on='id', how='inner')

In [69]:
Modelo2

,user_id,item_id,recommend,sentiment_analysis,genres,app_name,specs,price,id,developer,año
0,76561197970982479,1250.0,True,2,FPS,Killing floor,Single-player,19.99,1250.0,Tripwire interactive,2009
1,76561197970982479,1250.0,True,2,FPS,Killing floor,Multi-player,19.99,1250.0,Tripwire interactive,2009
2,76561197970982479,1250.0,True,2,FPS,Killing floor,Co-op,19.99,1250.0,Tripwire interactive,2009
3,76561197970982479,1250.0,True,2,FPS,Killing floor,Cross-Platform Multiplayer,19.99,1250.0,Tripwire interactive,2009
4,76561197970982479,1250.0,True,2,FPS,Killing floor,Steam Achievements,19.99,1250.0,Tripwire interactive,2009
...,...,...,...,...,...,...,...,...,...,...,...
7188340,vinquility,431510.0,True,2,Anime,Mystic destinies: serendipity of aeons,In-App Purchases,0.00,431510.0,Aeon dream studios,2016
7188341,vinquility,431510.0,True,2,Romance,Mystic destinies: serendipity of aeons,Single-player,0.00,431510.0,Aeon dream studios,2016
7188342,vinquility,431510.0,True,2,Romance,Mystic destinies: serendipity of aeons,Steam Achievements,0.00,431510.0,Aeon dream studios,2016
7188343,vinquility,431510.0,True,2,Romance,Mystic destinies: serendipity of aeons,Steam Trading Cards,0.00,431510.0,Aeon dream studios,2016


In [70]:
Modelo2['app_name'].value_counts()

app_name
Team fortress 2                                886560
Counter-strike: global offensive               593280
Garry's mod                                    374880
Dota 2                                         370560
Unturned                                       226820
                                                ...  
Cheats 4 hire                                       1
Dimmdrive :: gaming ramdrive @ 10,000+ mb/s         1
New york taxi simulator                             1
Get rich or die gaming                              1
Demonlisher                                         1
Name: count, Length: 3168, dtype: int64

Nos quedamos solamente con las columnas que vamos a utilizar

In [71]:
Modelo2 = Modelo2[['id','user_id', 'app_name', 'genres', 'developer', 'specs']]

In [72]:
# Limpieza y preprocesamiento de texto
Modelo2['combined_features'] = Modelo2['genres'] + ' ' + Modelo2['developer'] + ' ' + Modelo2['specs']

C:\Users\belen\AppData\Local\Temp\ipykernel_12800\1177131261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Modelo2['combined_features'] = Modelo2['genres'] + ' ' + Modelo2['developer'] + ' ' + Modelo2['specs']


In [73]:
Modelo2.shape

(7188345, 7)

In [74]:
Modelo2

,id,user_id,app_name,genres,developer,specs,combined_features
0,1250.0,76561197970982479,Killing floor,FPS,Tripwire interactive,Single-player,FPS Tripwire interactive Single-player
1,1250.0,76561197970982479,Killing floor,FPS,Tripwire interactive,Multi-player,FPS Tripwire interactive Multi-player
2,1250.0,76561197970982479,Killing floor,FPS,Tripwire interactive,Co-op,FPS Tripwire interactive Co-op
3,1250.0,76561197970982479,Killing floor,FPS,Tripwire interactive,Cross-Platform Multiplayer,FPS Tripwire interactive Cross-Platform Multip...
4,1250.0,76561197970982479,Killing floor,FPS,Tripwire interactive,Steam Achievements,FPS Tripwire interactive Steam Achievements
...,...,...,...,...,...,...,...
7188340,431510.0,vinquility,Mystic destinies: serendipity of aeons,Anime,Aeon dream studios,In-App Purchases,Anime Aeon dream studios In-App Purchases
7188341,431510.0,vinquility,Mystic destinies: serendipity of aeons,Romance,Aeon dream studios,Single-player,Romance Aeon dream studios Single-player
7188342,431510.0,vinquility,Mystic destinies: serendipity of aeons,Romance,Aeon dream studios,Steam Achievements,Romance Aeon dream studios Steam Achievements
7188343,431510.0,vinquility,Mystic destinies: serendipity of aeons,Romance,Aeon dream studios,Steam Trading Cards,Romance Aeon dream studios Steam Trading Cards


In [75]:
Modelo2['app_name'].value_counts()

app_name
Team fortress 2                                886560
Counter-strike: global offensive               593280
Garry's mod                                    374880
Dota 2                                         370560
Unturned                                       226820
                                                ...  
Cheats 4 hire                                       1
Dimmdrive :: gaming ramdrive @ 10,000+ mb/s         1
New york taxi simulator                             1
Get rich or die gaming                              1
Demonlisher                                         1
Name: count, Length: 3168, dtype: int64

Revisamos si hay valores nulos

In [76]:
Modelo2.isna().sum()

id                     0
user_id                0
app_name               0
genres                 0
developer            174
specs                  0
combined_features    174
dtype: int64

Eliminamos los valores nulos encontrados

In [77]:
Modelo2=Modelo2.dropna()

Agrupamos la columna 'id'

In [78]:
Modelo2 = Modelo2.groupby('id').first().reset_index()

Nos quedamos solamente con los 'id' distintos a 0

In [79]:
Modelo2 = Modelo2[Modelo2['id']!=0]

In [80]:
Modelo2['app_name'].nunique()

3166

In [81]:
Modelo2

,id,user_id,app_name,genres,developer,specs,combined_features
0,10.0,Bennysaputra,Counter-strike,Action,Valve,Multi-player,Action Valve Multi-player
1,20.0,76561198039441595,Team fortress classic,Action,Valve,Multi-player,Action Valve Multi-player
2,30.0,agenteagle,Day of defeat,FPS,Valve,Multi-player,FPS Valve Multi-player
3,40.0,2ez2xl,Deathmatch classic,Action,Valve,Multi-player,Action Valve Multi-player
4,50.0,pleasedontvotefortrump,Half-life: opposing force,FPS,Valve,Single-player,FPS Valve Single-player
...,...,...,...,...,...,...,...
3161,521340.0,76561198009992783,True or false,Casual,Vladimir maslov,Single-player,Casual Vladimir maslov Single-player
3162,521430.0,76561198075141715,Super switch,Indie,Red splat games,Single-player,Indie Red splat games Single-player
3163,521570.0,TfhuAWGscvg,You have 10 seconds 2,Free to Play,Tamationgames,Single-player,Free to Play Tamationgames Single-player
3164,521990.0,mikeyg74,Galactic storm,Action,Dawid bujak,Single-player,Action Dawid bujak Single-player


In [82]:
with gzip.open('./Modelo2.csv.gz', 'wb') as f:
    Modelo2.to_csv(f, index=False, encoding='utf-8')

Si es un sistema de recomendación user-item:

def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

Se preparan los datos de la columna 'combined_features' para ser utilizados en un modelo de machine learning al convertirlos en una matriz de recuento de términos

In [83]:
CV2 = CountVectorizer()
converted_metrix2 = CV2.fit_transform(Modelo2['combined_features'])

In [84]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [85]:
cosine_similarity = cosine_similarity(converted_metrix2)

In [88]:
import joblib #Se guarda la matriz

joblib.dump(cosine_similarity, 'Modelo2.pkl', compress=1)

['Modelo2.pkl']

In [89]:
import joblib
with open('Modelo2.pkl', 'rb') as file:
        modelo2 = joblib.load(file)


In [91]:
def recomendacion4(user_id):
    #Cargar el modelo entrenado desde el archivo pickle
    with open('Modelo1.pkl', 'rb') as file:
       modelo2 = joblib.load(file)

    Modelo2 = pd.read_csv('./Modelo2.csv.gz', compression='gzip')
    
    if user_id not in Modelo2['user_id'].tolist():
       return {"Respuesta": "No se encontraron resultados para la búsqueda realizada"}

    def get_recommendations_by_user(user_id, cosine_sim=modelo2):
    # Obtener los juegos jugados por el usuario
        games_played = Modelo2[Modelo2['user_id'] == user_id]['app_name'].tolist()

        # Inicializar la lista de juegos recomendados
        recommended_games = []

        # Iterar sobre los juegos jugados por el usuario
        for app_name in games_played:
            # Obtener el índice del juego actual
            idx = Modelo2[Modelo2['app_name'] == app_name].index[0]

            # Obtener los puntajes de similitud para el juego actual
            sim_scores = list(enumerate(cosine_sim[idx]))

            # Ordenar los juegos por similitud
            sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

            # Seleccionar los juegos más similares (excluyendo el juego actual)
            sim_scores = sim_scores[1:6]

            # Obtener los índices de los juegos recomendados
            game_indices = [i[0] for i in sim_scores]

            # Agregar los juegos recomendados a la lista
            recommended_games.extend(Modelo2['app_name'].iloc[game_indices])

         # Eliminar duplicados y devolver los primeros 5 juegos únicos recomendados
        return list(set(recommended_games))[:5]
  
    recommendations2 = get_recommendations_by_user(user_id)
    return {"Recomendaciones": recommendations2}

    

In [92]:
recomendacion4('Bennysaputra')

{'Recomendaciones': ["Macguffin's curse",
  'Call of duty®: black ops ii - apocalypse',
  'The howler',
  'Defy gravity extended',
  'X-com: interceptor']}